# UNDER CONSTRUCTION
- description wrong
- untested
- plot would be nice

# Description

This notebook calculates and plots the binomial distribution for a given single trial success probability ```p_success``` and a given number of trials ```n_trials```. Assuming there is a known, constant single trial success probability ```p_success``` between 0 (never successful) and 1 (always successful), this tells you how probable it is that you get 0, 1, ..., or ```n_trial``` succcessful results when conducting ```n_trial``` tests.

Note: all probabilities are given as numbers between 0 and 1, where 0 represents "never" and 1 represents 100% certainty.

# Parameters

In [36]:
n_trials_A = 5000 #number of trials conducted
n_successes_A = 5000 #number of successes

n_trials_B = 230
min_successes_B = 200

round_to_digits = 4

assert isinstance(n_trials_A, int) and n_trials_A > 0
assert isinstance(n_trials_B, int) and n_trials_B > 0
assert isinstance(n_successes_A, int) and n_successes_A <= n_trials_A
assert isinstance(min_successes_B, int) and 0 < min_successes_B <= n_trials_A
assert isinstance(round_to_digits, int) and round_to_digits >= 1

# Evaluation

In [37]:
import matplotlib.pyplot as plt
import mpmath as mp
import numpy as np
import scipy.optimize as opt

mp.mp.dps = 50 #use 50 decimal digits precision in mpmath calculations

def binomial_distr(n_successes, p_success, n_trials):
    return mp.binomial(n_trials, n_successes) * mp.power(mp.mpf(p_success), n_successes) * mp.power(mp.mpf(1-p_success), (n_trials-n_successes))

binomial_distr_vectorized = np.vectorize(binomial_distr) #create version of binomial distribution that can be evaluated for whole lists of inputs

def prob_null_hypothesis(p_success, n_trials_A, n_successes_A, n_trials_B, min_successes_B):
    return float( binomial_distr(n_successes_A, p_success, n_trials_A) * binomial_distr_vectorized(np.arange(0, min_successes_B), p_success, n_trials_B).sum() )

prob_null_hypothesis_vectorized = np.vectorize(prob_null_hypothesis)

prob = lambda p: -prob_null_hypothesis_vectorized(p, n_trials_A, n_successes_A, n_trials_B, min_successes_B)

max_probability_null_hypothesis = -opt.minimize(prob, n_successes_A/n_trials_A).fun
max_probability_null_hypothesis_rounded = mp.nstr(mp.mpf(max_probability_null_hypothesis), round_to_digits)
min_probability_hypothesis_rounded = mp.nstr(mp.mpf(1-max_probability_null_hypothesis), round_to_digits)

print('You have conducted {} trials in test (A) of which {} were successful'.format(n_trials_A, n_successes_A))
print('The probability of getting fewer than {} successful trials out of {} in test (B) is at most {}'.format(min_successes_B, n_trials_B, max_probability_null_hypothesis_rounded))
print('Thefore, the confidence is {} that you will get at least {} successful trials out of {} in test (B)'.format(min_probability_hypothesis_rounded, min_successes_B, n_trials_B))

You have conducted 5000 trials in test (A) of which 5000 were successful
The probability of getting fewer than 200 successful trials out of 230 in test (B) is at most 0.0
Thefore, the confidence is 1.0 that you will get at least 200 successful trials out of 230 in test (B)
